In [ ]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import SimpleITK as sitk
import time

In [ ]:
import ct_projector.projector.cupy as ct_projector
import ct_projector.projector.cupy.fan_equiangular as ct_fan
import ct_projector.projector.cupy.parallel as ct_para
import ct_projector.projector.numpy as numpy_projector
import ct_projector.projector.numpy.parallel as numpy_para

In [ ]:
# load a sample CT image
filename = './3.nii.gz'
ct = sitk.ReadImage(filename)
spacing = ct.GetSpacing()
img = sitk.GetArrayFromImage(ct)

# convert image from HU to attenuation coefficient
# This is the approximate relationship
img = (img.astype(np.float32) + 1024) / 1000 * 0.019
img[img < 0] = 0

# also convert to image to our projector dimension batch, z, y, x
img = img[np.newaxis, ...]
img = img[:, ::-1, ...]
spacing = np.array(spacing[::-1])

In [ ]:
# show the ct images
plt.figure(figsize = (12,4))
plt.subplot(131); plt.imshow(img[0, img.shape[1]//2, ...], 'gray', aspect=spacing[1] / spacing[2])
plt.subplot(132); plt.imshow(img[0, :, img.shape[2]//2, :], 'gray', aspect=spacing[0] / spacing[2])
plt.subplot(133); plt.imshow(img[0, ..., img.shape[3]//2], 'gray', aspect=spacing[0] / spacing[1])

In [ ]:
# setup the projector
projector = ct_projector.ct_projector()
projector.from_file('./projector_fan.cfg')
projector.nx = img.shape[3]
projector.ny = img.shape[2]
projector.nz = 1
projector.nv = 1
projector.dx = spacing[2]
projector.dy = spacing[1]
projector.dz = spacing[0]
projector.nview = 768
# mimic parallel with fan beam, to compare with the tru parallel beam
projector.dsd += 10000
projector.dso += 10000

for k in vars(projector):
    print (k, '=', getattr(projector, k))

In [ ]:
# very important: make sure that the arrays are saved in C order
cp.cuda.Device(0).use()
ct_projector.set_device(0)

angles = projector.get_angles()
# origin_img = img[0, [img.shape[1]//3, img.shape[1]//2, img.shape[1]//3*2], ...]
origin_img = img[0, :64, ...]
origin_img = origin_img[:, np.newaxis, ...]
cuimg = cp.array(origin_img, cp.float32, order = 'C')
cuangles = cp.array(angles, cp.float32, order = 'C')

In [ ]:
# set which device to use
numpy_projector.set_device(0)

# forward projection
projector.set_projector(ct_para.distance_driven_fp, angles = cuangles)
start = time.time()
cufp = projector.fp(cuimg, angles = cuangles)
end = time.time()
print (end - start)

projector.set_projector(ct_fan.distance_driven_fp, angles = cuangles)
start = time.time()
cufp_fan = projector.fp(cuimg, angles = cuangles)
end = time.time()
print (end - start)

In [ ]:
# show the projections
fp = cufp.get()
fp_fan = cufp_fan.get()
plt.figure(figsize = (16,8))
plt.subplot(121)
plt.imshow(fp[1,:, 0, :], 'gray', vmin=0, vmax=10)
plt.subplot(122)
plt.imshow(fp_fan[1,:, 0, :], 'gray', vmin=0, vmax=10)

In [ ]:
fbp_projector = numpy_projector.ct_projector()
fbp_projector.from_file('./projector_fan.cfg')
fbp_projector.nx = img.shape[3]
fbp_projector.ny = img.shape[2]
fbp_projector.nz = 1
fbp_projector.nv = 1
fbp_projector.dx = spacing[2]
fbp_projector.dy = spacing[1]
fbp_projector.dz = spacing[0]
fbp_projector.dsd += 10000
fbp_projector.dso += 10000

In [ ]:
fprj = numpy_para.ramp_filter(fbp_projector, fp, filter_type='RL')

In [ ]:
cufprj = cp.array(fprj, cp.float32, order='C')
projector.set_backprojector(ct_para.distance_driven_bp, angles=cuangles, is_fbp=True)
curecon = projector.bp(cufprj)
recon = curecon.get()

In [ ]:
plt.figure(figsize = (16,8))
plt.subplot(121); plt.imshow(recon[2, 0, :, :], 'gray', vmin=0, vmax=0.0225)
plt.subplot(122); plt.imshow(cuimg.get()[2, 0, :, :], 'gray', vmin=0, vmax=0.0225)